In [1]:
import MovementSimulator_0_1 as ms
import cv2
import path_tracking_ini_functions as ptif
import pidcontrol
import time
import math
import numpy as np

In [22]:
def reaction_f(action):
    x_a,y_a,f_a = action
    
    x_re = (x_a + 0.1*y_a) * f_a
    y_re = (y_a - 0.1*x_a) * f_a
    
    return x_re, y_re

def reaction_0(action):
    
    return 0,0

In [23]:
def generate_field_example():
    particle1 = ms.SimulatedParticleClass((30,30),reaction_f,25)

    
    target1 = ms.SimulatedTargetsClass((125,75), reaction_0,20)
    target2 = ms.SimulatedTargetsClass((180,250),reaction_0, 30)
    target3 = ms.SimulatedTargetsClass((330,310), reaction_0,25)
    target4 = ms.SimulatedTargetsClass((200,200), reaction_0,20)
    target5 = ms.SimulatedTargetsClass((300,100),reaction_0, 30)
    target6 = ms.SimulatedTargetsClass((300,40), reaction_0,25)
    
    particles_dict = {'p1':particle1}
    targets_dist = {}
    #targets_dist = {'t1':target1, 't2':target2, 't3':target3, 't4':target4, 't5':target5, 't6':target6}
    borders = (0,0,480,640)
    
    RealField = ms.SimulatedFieldClass(particles_dict,targets_dist,borders)
    return RealField

In [24]:
def vector_len(x):
    length = math.sqrt(x[0]**2 + x[1]**2)
    return length


def dot_star(v1,v2):
    dot = v1[0]*(v2[1])-v1[1]*v2[0]
    return dot

def vectors_angle_rad(v1,v2):
    c = np.dot(v1,v2)/(vector_len(v2)* vector_len(v2))
    return math.acos(c)

In [25]:
def show_path(image, path):
    
    for i in range(len(path)-1):
        image = cv2.line(image,path[i],path[i+1],(150,150,150))
    return image

In [28]:
# main tracker-controller cell

# generate simulation field
RF = generate_field_example()
image = RF.generate_image(width = 640, height = 480)



# cv2.imshow("Tracking", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# activate tracker
bound = cv2.selectROI(image)
tracker = cv2.legacy.TrackerCSRT_create()
ok = tracker.init(image, bound)
cv2.destroyAllWindows()

# define the target point
path_arr = [(100,100),(50,200),(400,400)]
p_step = 0
path = path_arr[p_step]

# motor position. [1,0] means that zero angle of the motor is parralel to Ox
motor_zero_vector = np.array([1,0])


# current coordinates of the particle
x0, y0 = (bound[0]+bound[2]//2,bound[1]+bound[3]//2)

# direction to the target
target_dir = (path[0]-x0,path[1]-y0)
target_dir_x, target_dir_y = (target_dir[0]/vector_len((target_dir[0],target_dir[1])),target_dir[1]/vector_len((target_dir[0],target_dir[1])))
target_dir_norm = np.array([target_dir_x,target_dir_y])


motor_target_angle = math.degrees(math.acos(np.dot(motor_zero_vector,target_dir_norm)/(vector_len(motor_zero_vector)*vector_len(target_dir_norm))))
dot_st = dot_star(motor_zero_vector,target_dir_norm)
print(motor_target_angle)
print(dot_st)

pid = pidcontrol.PIDClass((-100,0,0), motor_target_angle, (0,180), integration_samples = 5, diff_filter_samples = 4)

mark = True
time  = 0

while mark:
    force = 500
    action = (target_dir_x,target_dir_y,force)
    time_step = 2
    
    # make 10 timesteps of movement
    for i in range(10):
        ok = RF.next_time_step(action, time_step)
    
    image = RF.generate_image(width = 640, height = 480)
    ok, bound = tracker.update(image)
    if ok:
        p1 = (int(bound[0]), int(bound[1]))
        p2 = (int(bound[0] + bound[2]), int(bound[1] + bound[3]))
        image = cv2.rectangle(image, p1, p2, (150,150,150), 2, 1)
        image = show_path(image, path_arr)
        cv2.imshow("image", image)
        cv2.waitKey(300)
    
    
    # current coordinates of the particle
    x1, y1 = (bound[0]+bound[2]//2,bound[1]+bound[3]//2)

    # direction to the target
    target_dir = (path[0]-x1,path[1]-y1)
    target_dir_x, target_dir_y = (target_dir[0]/vector_len((target_dir[0],target_dir[1])),target_dir[1]/vector_len((target_dir[0],target_dir[1])))
    target_dir_norm = np.array([target_dir_x,target_dir_y])
    
    target_motor_angle = vectors_angle_rad(motor_zero_vector,target_dir_norm)
    
    real_dir = (x1-x0,y1-y0)
    real_dir_motor_vector = vectors_angle_rad(motor_zero_vector,real_dir)
    
    error = real_dir_motor_vector - target_motor_angle
    
    time = time_step * 10 + time
    
    new_angle = pid.control(time, real_dir_motor_vector)
    print('new angle'+ str(new_angle))
    pid.setpoint(target_motor_angle)
    
    
    target_dir_x = motor_zero_vector[0]*math.cos(new_angle)-motor_zero_vector[1]*math.sin(new_angle)
    target_dir_y = motor_zero_vector[1]*math.cos(new_angle)+motor_zero_vector[0]*math.sin(new_angle)
    targetdirlen = vector_len((target_dir_x,target_dir_y))
    target_dir_x = target_dir_x/targetdirlen
    target_dir_y = target_dir_y/targetdirlen
    target_dir_norm = np.array([target_dir_x,target_dir_y])
    
    target_dir_norm = np.array([target_dir_x,target_dir_y])

    motor_target_angle = math.degrees(math.acos(np.dot(motor_zero_vector,target_dir_norm)/(vector_len(motor_zero_vector)*vector_len(target_dir_norm))))
    dot_st = dot_star(motor_zero_vector,target_dir_norm)
    print(motor_target_angle)
    print(dot_st)
    
    if vector_len(target_dir) < 10:
        p_step = p_step + 1
        if (p_step<(len(path_arr))):
            path = path_arr[p_step]
        else:
            mark = False
            print('Target reached')
    
    if dot_st < 0:
        target_dir_x, target_dir_y = motor_zero_vector
        print('Out of regulation range')
    
    
    x0 = x1
    y0 = y1

cv2.waitKey(0)
cv2.destroyAllWindows()  
    

# turn vector on some angle
# x1=x*cos(angle)-y*sin(angle);
# y1=y*cos(angle)+x*sin(angle);
    
    

(0, 0, 640, 480)
43.745548377318464
0.6914569292855351
Real position
(30.19360531083621, 30.15145135137122)
Real position
(30.387210621672423, 30.302902702742443)
Real position
(30.580815932508635, 30.454354054113665)
Real position
(30.774421243344847, 30.605805405484887)
Real position
(30.96802655418106, 30.75725675685611)
Real position
(31.16163186501727, 30.90870810822733)
Real position
(31.355237175853482, 31.06015945959855)
Real position
(31.548842486689693, 31.211610810969773)
Real position
(31.742447797525905, 31.363062162340995)
Real position
(31.936053108362117, 31.514513513712217)
new angle180
126.75968764518224
-0.8011526357338306
Out of regulation range
Real position
(32.18063909562326, 31.490054914986104)
Real position
(32.42522508288441, 31.46559631625999)
Real position
(32.669811070145556, 31.441137717533877)
Real position
(32.9143970574067, 31.416679118807764)
Real position
(33.15898304466785, 31.39222052008165)
Real position
(33.403569031928996, 31.367761921355537)
Rea

new angle0
0.0
0.0
Real position
(66.42267731218377, 28.065851093330252)
Real position
(66.66726329944491, 28.04139249460414)
Real position
(66.91184928670606, 28.016933895878026)
Real position
(67.1564352739672, 27.992475297151913)
Real position
(67.40102126122835, 27.9680166984258)
Real position
(67.6456072484895, 27.943558099699686)
Real position
(67.89019323575064, 27.919099500973573)
Real position
(68.13477922301179, 27.89464090224746)
Real position
(68.37936521027294, 27.870182303521347)
Real position
(68.62395119753408, 27.845723704795233)
new angle112.4690777851493
35.996517182381034
-0.5877360737860647
Out of regulation range
Real position
(68.86853718479523, 27.82126510606912)
Real position
(69.11312317205638, 27.796806507343007)
Real position
(69.35770915931752, 27.772347908616894)
Real position
(69.60229514657867, 27.74788930989078)
Real position
(69.84688113383982, 27.723430711164667)
Real position
(70.09146712110096, 27.698972112438554)
Real position
(70.33605310836211, 2

new angle20.824876799620128
113.17754949813826
0.9192896289892147
Real position
(93.89568241103092, 28.954617331615665)
Real position
(93.8219023712282, 29.18908915069605)
Real position
(93.74812233142549, 29.42356096977643)
Real position
(93.67434229162278, 29.658032788856815)
Real position
(93.60056225182007, 29.892504607937198)
Real position
(93.52678221201735, 30.12697642701758)
Real position
(93.45300217221464, 30.361448246097964)
Real position
(93.37922213241193, 30.595920065178348)
Real position
(93.30544209260921, 30.83039188425873)
Real position
(93.2316620528065, 31.064863703339114)
new angle0
0.0
0.0
Real position
(93.47624804006765, 31.040405104613)
Real position
(93.72083402732879, 31.015946505886888)
Real position
(93.96542001458994, 30.991487907160774)
Real position
(94.21000600185108, 30.96702930843466)
Real position
(94.45459198911223, 30.942570709708548)
Real position
(94.69917797637338, 30.918112110982435)
Real position
(94.94376396363452, 30.89365351225632)
Real pos

new angle64.72343825769435
108.3798477414527
0.948986974059189
Real position
(122.20262338339171, 34.51390023247403)
Real position
(122.14871257699015, 34.7537213182219)
Real position
(122.09480177058859, 34.99354240396977)
Real position
(122.04089096418703, 35.23336348971764)
Real position
(121.98698015778547, 35.47318457546551)
Real position
(121.9330693513839, 35.71300566121338)
Real position
(121.87915854498235, 35.95282674696125)
Real position
(121.82524773858079, 36.19264783270912)
Real position
(121.77133693217922, 36.43246891845699)
Real position
(121.71742612577766, 36.67229000420486)
new angle14.868696745481502
131.91357037599283
0.7441533507231959
Real position
(121.57224147595159, 36.8706380459674)
Real position
(121.42705682612551, 37.06898608772993)
Real position
(121.28187217629943, 37.26733412949247)
Real position
(121.13668752647335, 37.465682171255004)
Real position
(120.99150287664727, 37.66403021301754)
Real position
(120.8463182268212, 37.862378254780076)
Real posi

new angle112.5886121733263
29.14770123316036
-0.4870626652440297
Out of regulation range
Real position
(135.16017495785871, 47.885222537672824)
Real position
(135.40476094511988, 47.86076393894671)
Real position
(135.64934693238104, 47.8363053402206)
Real position
(135.8939329196422, 47.811846741494485)
Real position
(136.13851890690336, 47.78738814276837)
Real position
(136.38310489416452, 47.76292954404226)
Real position
(136.62769088142568, 47.738470945316145)
Real position
(136.87227686868684, 47.71401234659003)
Real position
(137.116862855948, 47.68955374786392)
Real position
(137.36144884320916, 47.665095149137805)
new angle85.92853637366053
116.65752605287172
-0.8937042275926861
Out of regulation range
Real position
(137.60603483047032, 47.64063655041169)
Real position
(137.85062081773148, 47.61617795168558)
Real position
(138.09520680499264, 47.591719352959466)
Real position
(138.3397927922538, 47.56726075423335)
Real position
(138.58437877951496, 47.54280215550724)
Real positi

new angle69.92931813828295
46.654793551248275
0.7272314195629911
Real position
(162.8854747256585, 49.76459405075759)
Real position
(163.07114373846318, 49.92567647034547)
Real position
(163.25681275126786, 50.08675888993336)
Real position
(163.44248176407254, 50.24784130952124)
Real position
(163.62815077687722, 50.40892372910913)
Real position
(163.8138197896819, 50.57000614869701)
Real position
(163.99948880248658, 50.731088568284896)
Real position
(164.18515781529126, 50.89217098787278)
Real position
(164.37082682809594, 51.053253407460666)
Real position
(164.55649584090062, 51.21433582704855)
new angle134.13565878395804
125.40713052770347
0.8150556971875236
Real position
(164.43472209310747, 51.427857916220994)
Real position
(164.3129483453143, 51.64138000539344)
Real position
(164.19117459752115, 51.85490209456588)
Real position
(164.069400849728, 52.06842418373832)
Real position
(163.94762710193484, 52.281946272910766)
Real position
(163.82585335414169, 52.49546836208321)
Real p

<ipython-input-24-d30ea303deda>:11: RuntimeWarning: invalid value encountered in double_scalars
  c = np.dot(v1,v2)/(vector_len(v2)* vector_len(v2))


ValueError: cannot convert float NaN to integer

In [8]:
import math
x,y = (1,0)
angle = math.radians(89)
x1=x*math.cos(angle)-y*math.sin(angle)
y1=y*math.cos(angle)+x*math.sin(angle)
x1,y1

(0.0174524064372836, 0.9998476951563913)

In [9]:
from math import sqrt
v1 = (1,0)
v2 = (1,1)
a = v2
b = v1
cos_ab = (a[0]*b[0]+a[1]*b[1])/(sqrt(a[0]**2+a[1]**2)*sqrt(b[0]**2+b[1]**2))
cos_ab

0.7071067811865475

In [14]:
from tiltplatform import TiltPlatformClass
plat = TiltPlatformClass(port_name = 'COM6', x_channel = 3, y_channel = 5, 
                 x_range_tup = (0,90), y_range_tup = (80,100))

In [16]:
plat.close_serial()

port released


In [15]:
plat.move('x', 0)

ok


In [56]:
v = (1,0)

In [57]:
n = (-0.9,1)

In [58]:
np.dot(n,v)

-0.9

In [59]:
np.dot(v,n)

-0.9

In [60]:
dot_star(v,n)

1.0

In [61]:
dot_star(n,v)

-1.0

In [48]:
path = [(1,2)]
curr_pos_point = [7,9]
target_dir = (path[0][0]-curr_pos_point[0],path[0][1]-curr_pos_point[1])
target_dir_norm = (target_dir[0]/vector_len(target_dir[0],target_dir[1]),target_dir[1]/vector_len(target_dir[0],target_dir[1]))



In [49]:
np.array(target_dir_norm)

array([-0.65079137, -0.7592566 ])

In [67]:
vector_len(1,2)

2.23606797749979

In [4]:
br = (0,1)

In [5]:
if br == (0,0):
    print('ok')
else:
    print('not ok')

not ok


In [13]:
class MC():
    def __init__(self):
        self.a = 'c'
        print('ini')
    
    def test(self,obj):
        if hasattr(self, str(obj)):
            print('a exists')
    def restart(self):
        self.__init__()

In [14]:
mc = MC()

ini


In [15]:
mc.test('a')

a exists


In [16]:
mc.test('b')

In [18]:
mc.restart()

ini
